### unbalancing solution

### gini coefficient

https://www.kaggle.com/batzner/gini-coefficient-an-intuitive-explanation

### preprocessing

filling

dummy

normalization

difficulty on preprocessing after cv

In [1]:
import warnings 
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import xgboost as xgb

from preprocessing import dataset
from sklearn import metrics

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def loadcv(path):
    cv = []
    i = 0
    while True:
        try:
            train = dataset(pd.read_csv(path+"train_X_"+str(i)+".csv"),
                    pd.read_csv(path+"train_y_"+str(i)+".csv"))
            valid = dataset(pd.read_csv(path+"valid_X_"+str(i)+".csv"),
                    pd.read_csv(path+"valid_y_"+str(i)+".csv"))
            cv.append([train, valid])
            print("cross validation set %d loaded"%i)
            i += 1
        except:
            return cv

In [3]:
path = "./cv/cv_"
cv = loadcv(path)

cross validation set 0 loaded
cross validation set 1 loaded
cross validation set 2 loaded
cross validation set 3 loaded
cross validation set 4 loaded


In [4]:
test = dataset(pd.read_csv(path+"test_X"+".csv"),
               pd.read_csv(path+"test_y"+".csv"))

In [5]:
def gini(y, pred):
    fpr, tpr, thr = metrics.roc_curve(y, pred, pos_label=1)
    g = 2 * metrics.auc(fpr, tpr) -1
    return g

def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred)

In [7]:
params = {'eta': 0.02, 
          'max_depth': 4, 
          'subsample': 0.9, 
          'colsample_bytree': 0.9, 
          'objective': 'binary:logistic', 
          'eval_metric': 'auc', 
          'seed': 99, 
          'silent': True
         }
result = []
cvresult = []
for i,data in enumerate(cv):
    print("training CV round %d"%i)
    watchlist = [(xgb.DMatrix(data[0].X, data[0].y), 'train'), (xgb.DMatrix(data[1].X, data[1].y), 'valid')]
    model = xgb.train(params, xgb.DMatrix(data[0].X, data[0].y), 5000,  watchlist, feval=gini_xgb, maximize=True, verbose_eval=50, early_stopping_rounds=200)
    cvresult.append(model.predict(xgb.DMatrix(data[1].X), ntree_limit=model.best_ntree_limit+45))
    result.append  (model.predict(xgb.DMatrix(test.X), ntree_limit=model.best_ntree_limit+45))

training CV round 0
[0]	train-gini:0.200667	valid-gini:0.187099
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 200 rounds.
[50]	train-gini:0.243489	valid-gini:0.224317
[100]	train-gini:0.249059	valid-gini:0.229365
[150]	train-gini:0.261369	valid-gini:0.240079
[200]	train-gini:0.277807	valid-gini:0.251068
[250]	train-gini:0.290025	valid-gini:0.26018
[300]	train-gini:0.298404	valid-gini:0.265779
[350]	train-gini:0.305718	valid-gini:0.269906
[400]	train-gini:0.311843	valid-gini:0.272175
[450]	train-gini:0.318135	valid-gini:0.274115
[500]	train-gini:0.323605	valid-gini:0.275487
[550]	train-gini:0.329003	valid-gini:0.276901
[600]	train-gini:0.333901	valid-gini:0.277651
[650]	train-gini:0.338569	valid-gini:0.278006
[700]	train-gini:0.342955	valid-gini:0.278381
[750]	train-gini:0.347134	valid-gini:0.278777
[800]	train-gini:0.351026	valid-gini:0.279093
[850]	train-gini:0.355711	valid-gini:0.279084
[900]	trai

In [20]:
stage1result = pd.DataFrame()

In [21]:
stage1result['xgboost'] = [i for one_validation in cvresult for i in one_validation]

In [22]:
stage1result.to_csv('./predictions/meta_train/xgboost.csv', index=False, float_format='%.5f')

In [23]:
test.y['target'] = np.mean(result,axis = 0)

In [24]:
test.y.to_csv('./predictions/meta_test//xgboost.csv', index=False, float_format='%.5f')

In [39]:
cvlabel = []
for i,data in enumerate(cv):
    cvlabel.append(data[1].y['target'].as_matrix())

In [41]:
stage1label = pd.DataFrame()
stage1label['target'] = [i for one_validation in cvlabel for i in one_validation]

In [45]:
stage1label.as_matrix()

array([[0],
       [0],
       [0],
       ..., 
       [0],
       [0],
       [0]])

In [47]:
gini(stage1label.as_matrix(), stage1result.as_matrix())

0.28346273880991424

### Dependencies

In [4]:
import warnings 
warnings.filterwarnings("ignore")
import pandas as pd
import numpy  as np
# from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
# import xgboost as xgb
from multiprocessing import cpu_count

In [5]:
from collections import namedtuple
dataset = namedtuple("dataset","X y")

### Loading Raw Data

In [27]:
train = pd.read_csv("./original/train.csv")
test  = pd.read_csv("./original/test.csv")

In [28]:
class dataset:
    def __init__(self, data = None, label = None):
        self.X = data
        self.y = label

### Preprocessing

In [29]:
train = dataset(train.drop("id",1).drop("target",1), train.target)
test  = dataset(test.drop("id",1), test.id)

In [30]:
train.X = train.X.replace(-1, np.nan)
test.X  = test .X.replace(-1, np.nan)

##### fillNA

In [31]:
train.X = train.X.fillna(-1)
test.X = test.X.fillna(-1)

##### Dummy Catagories

In [26]:
#Although it is better to do everything after splitting train-validation
#we extract dummylist first in order not to have different dimension for different sets
CATAGORY_THRES = 10
def dummylist(df):
    assert(isinstance(df,pd.core.frame.DataFrame))
    dummylist = []
    for columnname in list(df.columns):
        if columnname.endswith("cat") and len(list(df[columnname].unique())) < CATAGORY_THRES:
            dummylist.append(columnname)
    return dummylist
    return df    
def dummy(df, dummylist):
    assert(isinstance(df,pd.core.frame.DataFrame))
    for columnname in dummylist:
        newcolumn = pd.get_dummies(df[columnname],prefix = "1hot_"+columnname[:-4])
        df = pd.concat([df, newcolumn],axis = 1)
    for columnname in dummylist:
        df = df.drop(columnname, axis = 1)
    return df

In [27]:
mylist = dummylist(train.X)

In [28]:
train.X = dummy(train.X, mylist)
test.X  = dummy(test.X,  mylist)

In [29]:
KFOLDS = 5
folds = list(StratifiedKFold(n_splits=KFOLDS, shuffle=True, random_state=2016).split(train.X, train.y))

In [32]:
tmp = pd.DataFrame.copy(train.X.head(10))

In [33]:
tmp['ps_ind_01'][0] = 3

In [34]:
train.X.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,2,2.0,5,1.0,0.0,0,1,0,0,0,...,9,1,5,8,0,1,1,0,0,1
1,1,1.0,7,0.0,0.0,0,0,1,0,0,...,3,1,1,9,0,1,1,0,1,0
2,5,4.0,9,1.0,0.0,0,0,1,0,0,...,4,2,7,7,0,1,1,0,1,0
3,0,1.0,2,0.0,0.0,1,0,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,0,2.0,0,1.0,0.0,1,0,0,0,0,...,3,1,1,3,0,0,0,1,1,0


In [35]:
tmp

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,3,2.0,5,1.0,0.0,0,1,0,0,0,...,9,1,5,8,0,1,1,0,0,1
1,1,1.0,7,0.0,0.0,0,0,1,0,0,...,3,1,1,9,0,1,1,0,1,0
2,5,4.0,9,1.0,0.0,0,0,1,0,0,...,4,2,7,7,0,1,1,0,1,0
3,0,1.0,2,0.0,0.0,1,0,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,0,2.0,0,1.0,0.0,1,0,0,0,0,...,3,1,1,3,0,0,0,1,1,0
5,5,1.0,4,0.0,0.0,0,0,0,1,0,...,4,2,0,9,0,1,0,1,1,1
6,2,1.0,3,1.0,0.0,0,1,0,0,0,...,3,0,0,10,0,1,0,0,1,0
7,5,1.0,4,0.0,0.0,1,0,0,0,0,...,7,1,3,6,1,0,1,0,1,0
8,5,1.0,3,1.0,0.0,0,0,1,0,0,...,4,2,1,5,0,1,0,0,0,1
9,1,1.0,2,0.0,0.0,0,1,0,0,0,...,3,5,0,6,0,1,0,0,1,0


In [36]:
df = pd.DataFrame(np.arange(36).reshape(9,4),
                      columns=['A', 'B', 'C', 'D'])

In [43]:
train_idx, valid_idx = folds[0]
train_this_cut = dataset(
    train.X.loc[train_idx], train.y.loc[train_idx])
valid_this_cut = dataset(
    train.X.loc[valid_idx], train.y.loc[valid_idx])
print(train_this_cut.X.shape)
train_this_cut.X, NAmethod = Preprocessing.fillNA(train_this_cut.X)

(476169, 86)


In [44]:
Preprocessing.dummy(
            Preprocessing.addhead(train_this_cut.X, myheader), mydummylist).shape

(476179, 125)

In [51]:
myheader = pd.DataFrame.copy(train.X.head(10))

In [54]:
myheader.shape

(10, 86)